# Processing eROSITA data

This tutorial will teach you how to use DAXA to process raw eROSITA data into a science ready state using one line of Python code (or several lines, if you wish to have more control over the settings for each step). **This relies on there being an initialised (either manually before launching Python, or in your bash profile/rc) backend installation of the eROSITA Science Analysis Software System (eSASS), including accessible calibration files** - DAXA will check for such an installation, and will not allow processing to start without it.

**All DAXA processing steps will parallelise as much as possible - processes running on different ObsIDs/instruments/sub-exposures will be run simultaneously (if cores are available)**

## Import Statements

In [1]:
from daxa.mission import eRASS1DE, eROSITACalPV
from daxa.archive import Archive
from daxa.process.simple import full_process_erosita
from daxa.process.erosita.clean import flaregti
from daxa.process.erosita.assemble import cleaned_evt_lists

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

Here we create an archive of eRASS DR1 observations of the galaxy cluster Abell 3266:

In [2]:
ec = eROSITACalPV()
ec.filter_on_positions([[133.071, -1.025]])
er = eRASS1DE()
er.filter_on_positions([[133.071, -1.025]])

arch = Archive('eFEDSXCS-339', [ec, er], clobber=True)

/mnt/pact/dt237/code/PycharmProjects/DAXA/daxa/mission/base.py:1075: UserWarning: A field-of-view cannot be easily defined for eROSITACalPV and this number is the approximate half-length of an eFEDS section, the worst case separation - this is unnecessarily large for pointed observations, and you should make your own judgement on a search distance.
  fov = self.fov


## One-line solution

Though we provide individual functions that wrap the various steps required to reduce and prepare eROSITA data, and they can be used separately for greater control over the configuration parameters, we also include a one-line solution which executes the processing steps with default configuration.

We believe that the default parameters are adequate for most use cases, and this allows for users unfamiliar with the intricacies of eROSITA data to easily start working with it. Executing the following will automatically generate cleaned combined event lists for all telescope modules selected upon the original declaration of the mission.

fm00_300007_020_EventList_c001.fits

In [3]:
full_process_erosita(arch)

eRASS DE:1 - Generating final event lists: 100%|████████████████████████████████| 2/2 [00:01<00:00,  1.01it/s]


## General arguments for all processing functions

There are some arguments that all processing functions will take - they don't control the behaviour of the processing step itself, but instead how the commands are executed:

* **num_cores** - The number of cores that the function is allowed to use. This is set globally by the daxa.NUM_CORES constant (if set before any other DAXA function is imported), and by default is 90% of the cores available on the system.
* **timeout** - This controls how long an individual instance of the process is allowed to run before it is cancelled; the whole function may run for longer depending how many pieces of data need processing and how many cores are allocated. The default is generally null, but it can be set using an astropy quantity with time units.

## Breaking down the eROSITA processing steps

There are fewer individual steps for eROSITA compared to a telescope like XMM - this reflects its simpler design, with only a single camera type, as well as the differences in how data are served to the community and the backend software design. This section deals describes the different processing steps that DAXA can apply to eROSITA data (both all-sky and calibration).

### Identifying periods of high flaring (flaregti)

[flaregti function](../../daxa.process.erosita.html#daxa.process.erosita.clean.flaregti)

In [6]:
help(flaregti)

Help on function flaregti in module daxa.process.erosita.clean:

flaregti(obs_archive: daxa.archive.base.Archive, pimin: astropy.units.quantity.Quantity = <Quantity 200. eV>, pimax: astropy.units.quantity.Quantity = <Quantity 10000. eV>, mask_pimin: astropy.units.quantity.Quantity = <Quantity 200. eV>, mask_pimax: astropy.units.quantity.Quantity = <Quantity 10000. eV>, binsize: int = 1200, detml: Union[float, int] = 10, timebin: astropy.units.quantity.Quantity = <Quantity 20. s>, source_size: astropy.units.quantity.Quantity = <Quantity 25. arcsec>, source_like: Union[float, int] = 10, threshold: astropy.units.quantity.Quantity = <Quantity -1. ct / (deg2 s)>, max_threshold: astropy.units.quantity.Quantity = <Quantity -1. ct / (deg2 s)>, mask_iter: int = 3, num_cores: int = 115, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    The DAXA wrapper for the eROSITA eSASS task flaregti, which attempts to identify good time intervals with
    minimal flaring. 

### Applying event filtering and good-time-intervals to the raw event lists (cleaned_evt_lists)

This function ([cleaned_evt_lists function](../../daxa.process.erosita.html#daxa.process.erosita.assemble.cleaned_evt_lists)) creates the final, filtered and cleaned, event lists for eROSITA data. We make use of the evtool eSASS task for this. Our function will apply the good-time intervals generated by `flaregti`, as well as allowing the filtering of events based on pattern, flag, and energy. This is achieved through the passage of the following arguments:

* **lo_en** - This controls the lowest energy of event allowed into the cleaned event lists - the default is 0.2 keV, the lowest allowed by the eSASS tool.
* **hi_en** - This controls the highest energy of event allowed into the cleaned event lists - the default is 10.0 keV, the highest allowed by the eSASS tool.
* **flag** - Events are flagged during their initial processing (prior to download) - the flags represent combinations of circumstances, and include information on the owner (MPE or IKE), rejection decision, quality, and whether they are corrupted or not. We use a default value that will select all events flagged as either singly corrupt or as part of a corrupt frame.
* **flag_invert** - This controls whether the flag is used to define which events to _select_ or which to _exclude_. It is often easier to define the bad events with a flag and then invert it, which is the default behaviour here - any event selected by `flag` will be excluded, unless `flag_invert` is set to False.
* **pattern** - Defines which event patterns are acceptable (where a pattern describes how an event was registered by the detector ([this](https://erosita.mpe.mpg.de/edr/eROSITATechnical/patternfractions.html) discusses eROSITA pattern fractions. - the default value is 15, which represent 1111 in binary, which in turn means that single, double, triple, and quadruple events are all selected by default. If the absolute highest quality is required, and you have sufficient events, then it may make sense to limit this more, in which case you could pass 1000 (for singles only), or 1010 (for singles and triples), etc.

In [7]:
help(cleaned_evt_lists)

Help on function cleaned_evt_lists in module daxa.process.erosita.assemble:

cleaned_evt_lists(obs_archive: daxa.archive.base.Archive, lo_en: astropy.units.quantity.Quantity = <Quantity 0.2 keV>, hi_en: astropy.units.quantity.Quantity = <Quantity 10. keV>, flag: int = 3221225472, flag_invert: bool = True, pattern: int = 15, num_cores: int = 115, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    The function wraps the eROSITA eSASS task evtool, which is used for selecting events.
    This has been tested up to evtool v2.10.1
    
    This function is used to apply the soft-proton filtering (along with any other filtering you may desire, including
    the setting of energy limits) to eROSITA event lists, resulting in the creation of sets of cleaned event lists
    which are ready to be analysed.
    
    :param Archive obs_archive: An Archive instance containing eROSITA mission instances with observations for
        which cleaned event lists should be 